In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch
# Añade esto antes de cargar el modelo
from unsloth import is_bfloat16_supported
torch.set_default_dtype(torch.bfloat16 if is_bfloat16_supported() else torch.float16)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/roberto/Projects/ChartBot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-01 00:13:05.263816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740784385.281563   27474 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740784385.286991   27474 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-01 00:13:05.305793: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-

🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
from unsloth import FastLanguageModel
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from unsloth import is_bfloat16_supported
from unsloth import FastLanguageModel
from datasets import Dataset, DatasetDict, Features, Value
import glob
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import tensorflow as tf

max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",     
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",                  # probar la normal
    "unsloth/llama-3-8b-Instruct-bnb-4bit",         # probar la version que sabe seguir instrucciones code
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",     
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "Qwen/Qwen2.5-Coder-7B-Instruct",
    "unsloth/Qwen2.5-Coder-3B-bnb-4bit",
    "Qwen/Qwen2.5-Coder-3B-Instruct",
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    "Qwen/Qwen2.5-Coder-14B-Instruct",
    "unsloth/Qwen2.5-Coder-14B-Instruct"
]

# Configurar BitsAndBytes para cuantización en 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,       # Habilitar cuantización en 4 bit
    bnb_4bit_quant_type="nf4",       # nf4 recomendado para entrenamiento
    bnb_4bit_compute_dtype=torch.float16,  # Configurar cálculos en float16
    bnb_4bit_use_double_quant=True  # Usar doble cuantización
)
"""
# Nombre del modelo
model_name = "unsloth/Qwen2.5-Coder-7B-Instruct"

# Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Cargar el modelo con cuantización
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config,  # Configuración de BitsAndBytes
    device_map="auto"
)
model.gradient_checkpointing_enable()
"""
"""
# Carga el modelo con Unsloth (optimizado para entrenamiento)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-Coder-7B-Instruct",
    max_seq_length=max_seq_length,
    dtype=torch.bfloat16 if is_bfloat16_supported() else torch.float16,
    quantization_config = bnb_config,
    device_map = "auto"
)
"""
"""
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/Qwen2.5-Coder-7B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",  # auto-detecta GPU y, si es necesario, hace offload a CPU/RAM
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("unsloth/Qwen2.5-Coder-7B-Instruct", trust_remote_code=True)

"""
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/Qwen2.5-Coder-14B-Instruct-bnb-4bit",
    quantization_config=bnb_config,
    #device_map="auto"  # auto-detecta GPU y, si es necesario, hace offload a CPU/RAM
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("unsloth/Qwen2.5-Coder-14B-Instruct-bnb-4bit", trust_remote_code=True)



`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards:  50%|█████     | 1/2 [00:17<00:17, 17.19s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 34.00 MiB. GPU 0 has a total capacity of 9.77 GiB of which 76.88 MiB is free. Including non-PyTorch memory, this process has 8.75 GiB memory in use. Of the allocated memory 8.53 GiB is allocated by PyTorch, and 13.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
"""
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
-----------------------

model = prepare_model_for_kbit_training(
    model,
    use_gradient_checkpointing=True, 
    gradient_checkpointing_kwargs={"use_reentrant": False}
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 2,                  # Reduce a 2 (mínimo para mantener rendimiento)
    lora_alpha = 8,         # Alpha = 4x el rank
    target_modules = ["q_proj", "v_proj"],  
    lora_dropout = 0.1,     # Aumenta dropout para regularización
    use_gradient_checkpointing = "unsloth",  # Checkpointing optimizado
    bias = "none",
    max_seq_length = max_seq_length,
    random_state = 3407,
)


model = prepare_model_for_kbit_training(model)  # Añade esto antes de aplicar LoRA

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, lora_config) 
"""


# Aplica la configuración LoRA de Unsloth
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # Aumentar rango QLoRA ver indicaciones de la formula
    lora_alpha=32,
    #target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1, 
    use_gradient_checkpointing=True,  # Mejor integrado que enable()
)

# Verifica parámetros entrenables
model.print_trainable_parameters()



Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.1.6 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable params: 10,092,544 || all params: 7,625,709,056 || trainable%: 0.1323


In [ ]:
alpaca_prompt= """Below is an instruction that give a prompt. Write a response that appropriately completes the request and gives you a visualization code and the corresponding explanation.

### visualization _prompt:
{}

### visualization_code:
{}

### visualization_explanation:
{}"""

In [ ]:
dataset = 0
dataset_2 = 0

In [ ]:
def formatting_prompts_func(examples, tokenizer):
    visualization_prompts = examples["visualization_prompt"]
    visualization_codes = examples["visualization_code"]
    visualization_explanations = examples['visualization_explanation']
    
    texts = []
    EOS_TOKEN = tokenizer.eos_token  # Token de finalización

    for prompt, code_list, explanation in zip(visualization_prompts, visualization_codes, visualization_explanations):
        # Manejar diferentes niveles de anidamiento
        if isinstance(code_list, list):
            # Aplanar si es una lista de listas
            flattened_code_list = []
            for item in code_list:
                if isinstance(item, dict):
                    flattened_code_list.append(item)
                elif isinstance(item, list):
                    for sub_item in item:
                        if isinstance(sub_item, dict):
                            flattened_code_list.append(sub_item)
            # Extraer los códigos
            html_codes = [code.get("html", "") for code in flattened_code_list]
            css_codes = [code.get("CSS", "") for code in flattened_code_list]
            js_codes = [code.get("JavaScript", "") for code in flattened_code_list]
            
            html_code = "\n".join(html_codes)
            css_code = "\n".join(css_codes)
            js_code = "\n".join(js_codes)
        else:
            html_code = ""
            css_code = ""
            js_code = ""
        
        # Formatear usando la plantilla `alpaca_prompt`
        text = alpaca_prompt.format(
            prompt,
            f"HTML:\n{html_code}\n\nCSS:\n{css_code}\n\nJavaScript:\n{js_code}",
            explanation
        ) + EOS_TOKEN  # Añadir el token EOS al final de cada texto
        
        texts.append(text)

    return {"text": texts}


In [ ]:
def formatting_prompts_func_2(examples, tokenizer):
    visualization_prompt = examples["visualization_prompt"]
    visualization_code = examples["visualization_code"]      # Cadena de texto (JSON)
    visualization_explanation = examples["visualization_explanation"]

    texts = []
    EOS_TOKEN = tokenizer.eos_token

    for prompt, code, explanation in zip(visualization_prompt, visualization_code, visualization_explanation):
        # Convertir el objeto visualization_code en una cadena JSON formateada (si es necesario)
        try:
            json.loads(code)
            chart_options_str = code
        except json.JSONDecodeError:
            # Si no es un JSON válido, intenta convertirlo
            chart_options_str = json.dumps(code, indent=4)

        # Formatear usando la plantilla para los nuevos datos
        text = alpaca_prompt.format(
            prompt,
            chart_options_str,  # Ahora la cadena JSON formateada
            explanation
        ) + EOS_TOKEN

        texts.append(text)

    return {"text": texts}


In [ ]:
dataset = load_dataset("/home/roberto/Projects/ChartBot/dataset_test/", split = "train")

In [ ]:

# Ruta a los archivos del dataset
dataset_path = "/home/roberto/Projects/ChartBot/dataset_test_2/*.json"  # Ajusta la extensión si es necesario

# Paso 1: Leer y consolidar todos los archivos JSON
data = []
json_files = glob.glob(dataset_path)

for file_path in json_files:
    with open(file_path, 'r') as f:
        try:
            # Asumiendo JSON estándar
            entries = json.load(f)
            if isinstance(entries, list):
                data.extend(entries)
            elif isinstance(entries, dict):
                data.append(entries)
            else:
                print(f"Formato desconocido en {file_path}")
        except json.JSONDecodeError as e:
            print(f"Error al leer {file_path}: {e}")

# Paso 2: Verificar y transformar los datos
for entry in data:
    # Asegurarse de que todas las claves existan
    if "visualization_prompt" not in entry:
        entry["visualization_prompt"] = ""
    if "visualization_code" not in entry:
        entry["visualization_code"] = {}
    if "visualization_explanation" not in entry:
        entry["visualization_explanation"] = ""
    
    # Convertir `visualization_code` a cadena JSON si es un dict o list
    if isinstance(entry["visualization_code"], (dict, list)):
        entry["visualization_code"] = json.dumps(entry["visualization_code"])
    else:
        entry["visualization_code"] = str(entry["visualization_code"])
    
    # Asegurarse de que los demás campos sean cadenas
    entry["visualization_prompt"] = str(entry.get("visualization_prompt", ""))
    entry["visualization_explanation"] = str(entry.get("visualization_explanation", ""))

# Paso 3: Convertir la lista de diccionarios a un DataFrame de pandas
df = pd.DataFrame(data)

# Paso 4: Definir el esquema del dataset utilizando `Features`
features = Features({
    "visualization_prompt": Value("string"),
    "visualization_code": Value("string"),  # Ahora es una cadena JSON
    "visualization_explanation": Value("string")
})

# Paso 5: Crear el objeto Dataset desde el DataFrame
dataset_aux = Dataset.from_pandas(df, features=features, preserve_index=False)

# Paso 6: Crear DatasetDict si es necesario
dataset_2= DatasetDict({
    "train": dataset_aux
})

# Verificar el contenido
print(dataset_2)
print(dataset_2["train"][1])



DatasetDict({
    train: Dataset({
        features: ['visualization_prompt', 'visualization_code', 'visualization_explanation'],
        num_rows: 983
    })
})
{'visualization_prompt': "Create a Highcharts chartOptions object to render a 3D pie chart with the following specifications: Chart type: pie, Chart title: '3D Pie Chart Example', Axes and labels styling: Use a dark blue color (#003366) for the title and labels, with a font size of 14px for labels and 18px for the title. Data series: Include one series with the data [2, 4, 6, 1, 3]. Custom colors: Use a specific list of colors ['#FF6384', '#36A2EB', '#FFCE56', '#4BC0C0', '#9966FF']. Additional configurations: Set the chart margins to 50px, use a light gray background (#F5F5F5), and enable the legend with a dark gray color (#333333).", 'visualization_code': '{"chart": {"backgroundColor": "#F5F5F5", "margin": [50, 50, 50, 50]}, "title": {"text": "3D Pie Chart Example", "style": {"color": "#003366", "fontSize": "18px"}}, "plotOpt

In [ ]:
print(dataset[1])



{'visualization_prompt': 'Create a visualization of a chart with a logarithmic color axis that supports negative values. The chart should display monthly data points ranging from negative to positive values, with a custom axis conversion to handle negative logarithmic values.', 'visualization_code': [{'html': '<div id="container"></div>', 'CSS': '#container {\n    min-width: 300px;\n    max-width: 800px;\n    margin: 0 auto;\n}\n', 'JavaScript': "/**\n * Custom Axis extension to allow emulation of negative values on a logarithmic\n * color axis. Note that the scale is not mathematically correct, as a true\n * logarithmic axis never reaches or crosses zero.\n */\n(function (H) {\n    H.addEvent(H.Axis, 'afterInit', function () {\n        const logarithmic = this.logarithmic;\n\n        if (logarithmic && this.options.allowNegativeLog) {\n\n            // Avoid errors on negative numbers on a log axis\n            this.positiveValuesOnly = false;\n\n            // Override the converter 

In [ ]:
print(dataset_2["train"][1])

{'visualization_prompt': "Create a Highcharts chartOptions object to render a 3D pie chart with the following specifications: Chart type: pie, Chart title: '3D Pie Chart Example', Axes and labels styling: Use a dark blue color (#003366) for the title and labels, with a font size of 14px for labels and 18px for the title. Data series: Include one series with the data [2, 4, 6, 1, 3]. Custom colors: Use a specific list of colors ['#FF6384', '#36A2EB', '#FFCE56', '#4BC0C0', '#9966FF']. Additional configurations: Set the chart margins to 50px, use a light gray background (#F5F5F5), and enable the legend with a dark gray color (#333333).", 'visualization_code': '{"chart": {"backgroundColor": "#F5F5F5", "margin": [50, 50, 50, 50]}, "title": {"text": "3D Pie Chart Example", "style": {"color": "#003366", "fontSize": "18px"}}, "plotOptions": {"pie": {"depth": 45, "startAngle": -90, "endAngle": 90, "colors": ["#FF6384", "#36A2EB", "#FFCE56", "#4BC0C0", "#9966FF"]}}, "series": [{"data": [2, 4, 6,

In [ ]:
print(tokenizer)

Qwen2TokenizerFast(name_or_path='unsloth/qwen2.5-coder-7b-instruct-bnb-4bit', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|PAD_TOKEN|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, sin

In [ ]:
dataset_2 = dataset_2.map(formatting_prompts_func_2, batched=True, fn_kwargs={'tokenizer': tokenizer})

Map: 100%|██████████| 983/983 [00:00<00:00, 32029.78 examples/s]


In [ ]:
dataset = dataset.map(formatting_prompts_func, batched=True, fn_kwargs={'tokenizer': tokenizer})


In [ ]:
#print(dataset["text"][0])
print(dataset_2)

DatasetDict({
    train: Dataset({
        features: ['visualization_prompt', 'visualization_code', 'visualization_explanation', 'text'],
        num_rows: 983
    })
})


In [ ]:
# Dividir el dataset en entrenamiento y validación (90% train, 10% validation)
split_dataset = dataset_2["train"].train_test_split(test_size=0.1, seed=3407)
# Crear un nuevo DatasetDict con los splits 'train' y 'validation'
dataset_2 = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]  # 'test' se renombra como 'validation'
})
print(dataset_2)


DatasetDict({
    train: Dataset({
        features: ['visualization_prompt', 'visualization_code', 'visualization_explanation', 'text'],
        num_rows: 884
    })
    validation: Dataset({
        features: ['visualization_prompt', 'visualization_code', 'visualization_explanation', 'text'],
        num_rows: 99
    })
})


In [ ]:
# Verificar una muestra del split de entrenamiento
print(dataset_2["train"][0])

# Verificar una muestra del split de validación
print(dataset_2["validation"][0])


{'visualization_prompt': "Create a Highcharts chartOptions object to render a chart with the following specifications: Chart type: line and pie, Chart title: 'Series and point color index', Axes and labels styling: x-axis categories are ['Monday', 'Tuesday', 'Wednesday', 'Thursday'], y-axis title is null, Data series: 'Apples' series with data [0, -1, 1, 0], 'Oranges' series with data [1, 2, 6, 7], 'Chilis' series with data [1, 1, 3, 2], and a pie series named 'Totals' with data [{ name: 'Chilis', y: 6 }, { name: 'Oranges', y: 16 }], Custom colors: green for the first series, orange for the second, and red for the third, Additional configurations: styledMode enabled, pie series size is '30%' and centered at ['20%', '30%'].", 'visualization_code': '{"chart": {"styledMode": true}, "title": {"text": "Series and point color index"}, "xAxis": {"categories": ["Monday", "Tuesday", "Wednesday", "Thursday"]}, "yAxis": {"title": {"text": null}}, "series": [{"type": "line", "name": "Apples", "dat

In [ ]:
# Aplicar el formateo al split de entrenamiento
dataset_2["train"] = dataset_2["train"].map(
    formatting_prompts_func_2,
    batched=True,
    fn_kwargs={'tokenizer': tokenizer},
    remove_columns=["visualization_prompt", "visualization_code", "visualization_explanation"]
    if all(col in dataset_2["train"].column_names for col in ["visualization_prompt", "visualization_code", "visualization_explanation"])
    else None
)

# Aplicar el formateo al split de validación
dataset_2["validation"] = dataset_2["validation"].map(
    formatting_prompts_func_2,
    batched=True,
    fn_kwargs={'tokenizer': tokenizer},
    remove_columns=["visualization_prompt", "visualization_code", "visualization_explanation"]
    if all(col in dataset_2["validation"].column_names for col in ["visualization_prompt", "visualization_code", "visualization_explanation"])
    else None
)
print(dataset_2["train"].column_names)
print(dataset_2["validation"].column_names)


Map: 100%|██████████| 99/99 [00:00<00:00, 23220.90 examples/s]

['text']
['text']


In [ ]:
"""
# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    per_device_train_batch_size = 2,                # Tamaño del batch por dispositivo
    gradient_accumulation_steps = 1,                # Acumular gradientes durante 2 pasos
    warmup_steps = 190,                             # Número de pasos de warmup es entre un 1% y 10% del total de pasos
    num_train_epochs=3,                             #Descomenta y ajusta si prefieres usar épocas en lugar de max_steps
    #max_steps = 60,                                # Número máximo de pasos de entrenamiento
    learning_rate = 2e-5,                           # Tasa de aprendizaje
    fp16 = not is_bfloat16_supported(),             # Usar FP16 si BFLOAT16 no es soportado
    bf16 = is_bfloat16_supported(),                 # Usar BFLOAT16 si es soportado
    logging_steps = 50,                             # Registrar cada paso
    save_steps=500,                                 # Guarda el modelo cada 1000 pasos
    optim = "adamw_8bit",                           # Optimizador AdamW con 8-bit
    weight_decay = 0.01,                            # Decaimiento de peso
    lr_scheduler_type = "linear",                   # Tipo de scheduler de LR
    seed = 3407,                                    # Semilla para reproducibilidad
    output_dir = "outputs",                         # Directorio de salida
)

# Crear el entrenador
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset_2["train"],  # Usar el split de entrenamiento
    eval_dataset=dataset_2["validation"],  # Usar el split de validación
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,                       # Número de procesos para el dataset
    packing = False,                             # Desactivar packing si no es necesario
    args = training_args,
)
"""


# Definir los argumentos de entrenamiento ajustados
training_args = TrainingArguments(
    per_device_train_batch_size=1,                # Reducido de 2 a 1
    gradient_accumulation_steps=8,                # Ajustado para mantener el tamaño de batch efectivo
    warmup_steps=175,
    num_train_epochs=3,
    learning_rate=3e-5,
    fp16 = not is_bfloat16_supported(),  # Auto-ajustar según hardware
    bf16 = is_bfloat16_supported(),                                 # Desactivar BF16
    logging_steps=50,
    save_steps=500,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear", # antes --> cosine
    seed=3407,
    output_dir="outputs",
    gradient_checkpointing=False,
    max_grad_norm= 0.3  # Desactivar clipping de gradiente
)


# Crear el entrenador con los splits específicos y el data collator personalizado
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_2["train"],            # Usar el split de entrenamiento
    eval_dataset=dataset_2["validation"],        # Usar el split de validación
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

Map (num_proc=2): 100%|██████████| 99/99 [00:00<00:00, 138.85 examples/s]


In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3080. Max memory = 9.765 GB.
5.258 GB of memory reserved.


In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Parámetros entrenables: {trainable_params}")

Parámetros entrenables: 10092544


In [ ]:
# Iniciar el entrenamiento
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 884 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 330
 "-____-"     Number of trainable parameters = 10,092,544


Step,Training Loss
50,0.867800
100,0.801900
150,0.569900
200,0.447800
250,0.426700
300,0.418500


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2056.3963 seconds used for training.
34.27 minutes used for training.
Peak reserved memory = 6.648 GB.
Peak reserved memory for training = 1.39 GB.
Peak reserved memory % of max memory = 68.08 %.
Peak reserved memory for training % of max memory = 14.235 %.


In [ ]:
model.eval()  # Poner el modelo en modo de evaluación


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151665)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

In [ ]:

# Configurar el modelo en modo de inferencia
FastLanguageModel.for_inference(model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151665)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

def generate_visualization_code(prompt):
    # Tokenizar el prompt de entrada
    text = "It responds in JSON format with two attributes the name of the attributes must being visualization_code and visualization_explanation, in visualization_code it only writes a chartOptions object from HighchartJS and in visualization_explanation it only writes the chart's explanation.\n"
    """JSON example expected:\n"
        "{\n"
        '  "visualization_code": { /* chartOptions object */ },\n'
        '  "visualization_explanation": "Tu explicación aquí."\n'
        "}\n""" f"{prompt}\n" 
    "Response:"
    inputs = tokenizer(text, return_tensors="pt").to("cuda")  # Asegúrate de que el modelo esté en la GPU
    # Determinar la longitud máxima permitida
    max_seq_length = model.config.max_position_embeddings or 1024
    input_length = inputs.input_ids.shape[1]
    available_tokens = max_seq_length - input_length - 10  # Reservar algunos tokens

    max_new_tokens = min(800, available_tokens)  # Ajustar según tus necesidades
    # Generar la respuesta del modelo
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,            # Máxima longitud de la respuesta generada
            max_length = 1024,
            do_sample=False,                # Activar muestreo para generar variedad en las respuestas
            top_k=20,                      # Usar top-k sampling para controlar la diversidad
            top_p=0.9,                    # Usar nucleus sampling
            temperature=0.7,               # Controlar la creatividad en la generación
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decodificar el resultado a texto
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Response:" in response:
            response = response.split("Response:")[1].strip()
            
    return response

In [ ]:

def generate_visualization_code(prompt):
    system_message = (
        "It responds in JSON format with two attributes:\n"
        " - 'visualization_code': must contain only the chartOptions object from HighchartJS.\n"
        " - 'visualization_explanation': must contain only the explanation of the chart.\n\n"
        "Please follow exactly that JSON structure without extra text.\n\n"
    )
    
    # Construye el texto final donde incluyes tu prompt y la palabra "Response:" 
    text = f"{system_message}Prompt:\n{prompt}\n\nResponse:\n"
    
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    # Aquí, como quieres variedad, pon do_sample=True
    # Y ajusta tus parámetros de decodificación
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=920,
            do_sample=True,                # Para que haga muestreo
            top_k=40, 
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.2,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
        )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Si quieres cortar justo después de "Response:", lo puedes hacer.
    # En tu ejemplo, dices que luego parseas el JSON. 
    # Con que la salida sea consistente, basta.
    if "Response:" in response:
        response = response.split("Response:")[1].strip()

    return response


In [ ]:
# Ejemplo 1
prompt_1 = "Create a visualization that displays a stock chart for the company AAPL with a tooltip showing the stock price and a y-axis in percentage."

# Ejemplo 2
prompt_2 = "Generate a bar chart showing the sales performance of products A, B, and C with labels for each bar."

# Ejemplo 3
prompt_3 = "Create a line chart for website traffic over time, with data points for each week and tooltips displaying the visitor count."

In [ ]:
print("Respuesta para prompt 1:")
print(generate_visualization_code(prompt_1))


Respuesta para prompt 1:
{
  "visualization_code": {
    "tooltip": {
      "valueDecimals": 2,
      "pointFormat": "<b>{series.name}</b><br/>Price: ${y:.2f}"
    },
    "xAxis": {
      "type": "datetime",
      "labels": {
        "format": "{value:%B %e, '%Y'}"
      }
    },
    "plotOptions": {
      "area": {
        "fillOpacity": 0.5
      }
    },
    "rangeSelector": {
      "selected": 4
    },
    "title": {
      "text": "AAPL Stock Price"
    },
    "subtitle": {
      "text": "Source: <a href='https://www.alphavantage.co/'>Alpha Vantage</a>"
    },
    "legend": {
      "enabled": true
    },
    "credits": {
      "href": "http://www.highcharts.com/",
      "text": "\u00A9 Highsoft AS"
    },
    "series": [
      {
        "name": "AAPL",
        "data": [],
        "colorByPoint": false,
        "negativeColor": "#ff3d81",
        "zoneAxis": "y"
      }
    ]
  },
  "visualization_explanation": "The provided Highcharts configuration is designed to display a dynamic 

In [ ]:
print("\nRespuesta para prompt 2:")
print(generate_visualization_code(prompt_2))


Respuesta para prompt 2:
{
  "visualization_code": {
    "chart": {
      "type": "bar"
    },
    "title": {
      "text": "Sales Performance of Products A, B, and C",
      "style": {
        "fontSize": "18px"
      }
    },
    "xAxis": {
      "categories": ["Product A", "Product B", "Product C"],
      "labels": {
        "rotation": 0,
        "y": 36
      }
    },
    "yAxis": {
      "min": 0,
      "maxPadding": 0.2,
      "title": {
        "align": "high",
        "offset": 0,
        "floating": true,
        "formatters": null,
        "useHTML": false,
        "margin": 45,
        "radian": false,
        "gridLineWidth": 0,
        "left": null,
        "right": null,
        "top": null,
        "bottom": null,
        "textAlign": "",
        "verticalAlign": ""
      },
      "title": {
        "text": "Units sold",
        "style": {
          "color": "#000000",
          "fontWeight": "bold",
          "fontFamily": "'Open Sans', sans-serif"
        }
      }
 

In [ ]:
print("\nRespuesta para prompt 2:")
print(generate_visualization_code(prompt_2))


Respuesta para prompt 2:
{
  "visualization_code": {
    "chart": {
      "type": "bar"
    },
    "title": {
      "text": "Sales Performance of Products"
    },
    "xAxis": {
      "categories": ["Product A", "Product B", "Product C"],
      "labels": {
        "enabled": true
      }
    },
    "yAxis": {
      "min": 0,
      "maxPadding": 0.15,
      "tickInterval": 2,
      "minorTickLength": 3,
      "endOnTick": false,
      "showLastLabel": true
    },
    "legend": {
      "align": "right",
      "layout": "vertical",
      "backgroundColor": "#FFFFFF",
      "floating": true,
      "reversed": true,
      "shadow": true,
      "x": 400,
      "y": 70,
      "marker": {
        "symbol": "square"
      }
    },
    "credits": {
      "enabled": false
    },
    "plotOptions": {
      "series": {
        "stacking": null,
        "dataLabels": {
          "enabled": true,
          "format": "{point.y}%",
          "style": {
            "color": "(Highcharts.theme && Highch

In [ ]:
print("\nRespuesta para prompt 3:")
print(generate_visualization_code(prompt_3))


Respuesta para prompt 3:
{
  "visualization_code": {
    "title": {"text": "Website Traffic Over Time"},
    "xAxis": {"categories": ["Week 1", "Week 2", "Week 3", "Week 4"]},
    "yAxis": {"min": 0, "title": {"text": null}},
    "series": [{"name": "Visitor Count", "data": [500, 750, 689, 577]}],
    "tooltip": {"valueSuffix": "<br/>visitors"}
  },
  "visualization_explanation": "This line chart displays the website traffic over four weeks. The x-axis represents different weeks, while the y-axis shows the number of visitors. Each point on the line corresponds to the visitor count for that specific week."
}


In [ ]:
prompt_4 = """I want you to generate a bar visualisation for the following companies and how much on average it generates per year:
IBM = 30000000$.
Microsoft = 60000000$.
Tesla = 150000000$.
OpenAI = 200000000$."""
print("\nRespuesta para prompt 4:")
print(generate_visualization_code(prompt_4))


Respuesta para prompt 4:
{
"visualization_code": {
"xAxis": {"categories": ["IBM", "Microsoft", "Tesla", "OpenAI"]},
"title": {"text": "Average Revenue Per Year by Company"},
"yAxis": {"title": {"text": "Revenue ($)"}, "labels": {"format": "{value}"}},
"series": [{"data": [30000000, 60000000, 150000000, 200000000], "type": "bar"}]
},
"visualization_explanation": "This bar chart displays the average revenue generated by four major technology companies: IBM, Microsoft, Tesla, and OpenAI. The x-axis represents the companies, while the y-axis shows their respective revenues in dollars. Each company is represented by a bar, making it easy to compare their performance at a glance."
}


In [ ]:
prompt_5 = "HOW MUCH IT IS: 2 + 2 ?"

In [ ]:
print("\nRespuesta para prompt 5:")
print(generate_visualization_code(prompt_5))


Respuesta para prompt 5:
{
"visualization_code": {
    "title": {"text": "How much is 2 + 2?"},
    "xAxis": {"categories": ["Result"]},
    "yAxis": {"labels": {"format": "{value}"}},
    "series": [{"data": [4]}]
},
"visualization_explanation": "This bar chart shows the result of adding 2 and 2, which is 4."
}


In [ ]:
model.save_pretrained("./QLoRA_Qwenn_14B")
tokenizer.save_pretrained("./QLoRA_Qwenn_14B")

('./modelo_v1/tokenizer_config.json',
 './modelo_v1/special_tokens_map.json',
 './modelo_v1/vocab.json',
 './modelo_v1/merges.txt',
 './modelo_v1/added_tokens.json',
 './modelo_v1/tokenizer.json')

: 